## Camera Calibration

Cameras are mathematically modeled using intrinsic and extrinsic parameters. The intrinsic parameters include focal length, optical center, and distortion coefficients. Extrinsic parameters define the camera's position and orientation in space. If these parameters are known, images can be undistorted, and 3D points can be projected into the image plane accurately.

The process of estimating the camera parameters is called camera calibration. It is typically done using known patterns like chessboards.



### Step 1: **Capture Images**
Take multiple images of a known calibration pattern (e.g., chessboard) from different angles and distances. Use the following code to capture about 10 images from your webcam. Before running the code below, make sure you have a chessboard pattern printed out and visible to your webcam.

In [ ]:
# code is adapted from https://abauville.medium.com/display-your-live-webcam-feed-in-a-jupyter-notebook-using-opencv-d01eb75921d1
import cv2
import numpy as np

from IPython.display import display, Image
import time
import ipywidgets as widgets
import threading
import os

# Stop button
# ================
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

# Capture button
# ================
captureButton = widgets.ToggleButton(
    value=False,
    description='Capture',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='circle' # (FontAwesome names without the `fa-` prefix)
)

# Display function
# ================
def view(stopButton, captureButton=captureButton, filepath="./captures/"):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    while True:
        _, frame = cap.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value: # value changes to True when pressed
            cap.release()
            display_handle.update(None)
            stopButton.value = False
            break
        if captureButton.value:
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            filename = filepath + f"captured_{timestamp}.jpg"
            cv2.imwrite(filename, cv2.imdecode(frame, cv2.IMREAD_COLOR))
            print(f"Saved {filename}")
            captureButton.value = False
            continue
        time.sleep(0.03)  # ~30 FPS-ish; adjust as needed

# Run
# ================
display(stopButton)
display(captureButton)
# define and create the filepath where images will be saved
filepath = "./captures/"
if not os.path.exists(filepath):
    os.makedirs(filepath)
thread = threading.Thread(target=view, args=(stopButton,captureButton, filepath))
thread.start()

### Step 2: **Detect Corners**

Use OpenCV's `findChessboardCorners` function to detect the corners of the chessboard in each image you have captured. Store the detected corners for calibration and run the calibration process.

In [ ]:
import glob

chessboard_size = (9, 6)
image_glob = glob.glob(filepath + "*.jpg")

objp = np.zeros((np.prod(chessboard_size), 3), dtype=np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
object_points = []
image_points = []

for path in image_glob:
    gray = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY)
    found, corners = cv2.findChessboardCorners(gray, chessboard_size, None)
    if found:
        object_points.append(objp)
        refined = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        image_points.append(refined)

print(f"Frames with detected pattern: {len(image_points)} / {len(image_glob)}")
if image_points:
    _, camera_matrix, dist_coeffs, _, _ = cv2.calibrateCamera(object_points, image_points, gray.shape[::-1], None, None)
    print("Camera matrix:\n", camera_matrix)
    print("Distortion coefficients:\n", dist_coeffs.ravel())

## Question
Compare the intrinsic parameters obtained from your calibration with the default parameters of your camera. Also compare the results with other students. How consistent are the results? What factors could affect the accuracy of the calibration?

## Visualize the results
Now, we are able to undistort images from the camera using the obtained calibration parameters. Visualize the original and undistorted images side by side to see the effect of calibration.

In [ ]:
# Undistort button
# ================
undistortButton = widgets.ToggleButton(
    value=False,
    description='Undistort',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='circle' # (FontAwesome names without the `fa-` prefix)
)

# Display function
# ================
def view(stopButton, undistortButton=undistortButton):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    is_undistorted = False
    while True:
        _, frame = cap.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        if undistortButton.value:
            is_undistorted = not is_undistorted
            if is_undistorted:
                undistortButton.button_style = 'warning'
                undistortButton.description = 'Distort'
            else:
                undistortButton.button_style = 'info'
                undistortButton.description = 'Undistort'
            undistortButton.value = False
        if is_undistorted:
            frame = cv2.undistort(frame, camera_matrix, dist_coeffs)
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value: # value changes to True when pressed
            cap.release()
            display_handle.update(None)
            stopButton.value = False
            break
        
        time.sleep(0.03)  # ~30 FPS-ish; adjust as needed

# Run
# ================
display(stopButton)
display(undistortButton)

thread = threading.Thread(target=view, args=(stopButton, undistortButton))
thread.start()

## Question
How can you see if your calibration was successful? What visual cues indicate that the distortion has been corrected effectively?

## Bonus task

Use the code from the [OpenCV tutorial](https://docs.opencv.org/4.x/d7/d53/tutorial_py_pose.html) to estimate the pose of the chessboard in real-time using your webcam. Overlay a 3D cube on the chessboard to visualize the pose estimation.